# Описание решения

## TL;DR
0. Был использован лик, для контроля того, что процесс простроения решения идет в верном направлении.
1. Fine-tunning VGG16 (imagenet) для построения векторного представления постера (выбрал размерность 64).
2. CatBoost поверх категориальных признаков (язык, страна, класс фильма) и векторного представления постера.
3. Поиск оптимальных гипераметров CatBoost по 5-fold CV (предварительно, для каждого фолда была подготовлена своя VGG16).
4. Построение предсказаний на тесте как среднее по 5 моделям, полученных по итогу CV.

### 0. Лик
Так как конкурс проводится по открытым данным, то, безусловно, воспользоватся этим фактом просто необходимо.<br>
Мной было выполнено соотнесение тестовых данных с датасетом imdb5k. 90% строк были сматчены автоматически, остальные 10% потребовали ручной разметки. Задачу облегчило наличие постеров.

Полученая информация использовалась __исключительно__ для следующих целей:

* Оценка тестовой выборки.
* Оценка корректности проведения выбора гиперпараметров по честной кроссвалидации, основанной на обучающей выборке.

### 1. Описание решения
Решение состоит из 2 частей:

* Сверточная нейронная сеть для построения векторного представления постеров.
* Бинарный классификатор для построения предсказаний.

Так как бинарный классификатор требует выбора оптимального набора гиперпараметров, что удобно выполнять при помощи перекрестной проверки, то, соответственно, требуется обучить несколько нейросетевых моделей (по одной для каждого разбиения), для предотвращения "лика".

##### 1.1 Нейронная сеть
Были рассмотрены различные архитектуры нейросетевых моделей (InceptionV3, VGG16 и иные) для построения векторных представлений постеров. Лучше всех себя показала архитектура VGG16, ее скорость сходимости и финальные результаты оказались значительно лучше чем у иных рассмотренных.

Так как требовалось обучить сразу множество моделей, то было решено отказаться от ручного мониторинга сети (определения момента переобучения и т.д.). В качестве критерия останова был выбран механизм ранней остановки. Такая техника может показаться грубой, но оптимальная работа нейронной сети не является главной целью. Ее задача заключается в построении адекватного векторного представления постера, что может быть успешно выполнено как в несколько недообученном состоянии, так и в переобученном. Тем не менее история обучения сохранялась для построения кривых обучения.

##### 1.2 Бинарный классификатор
В качестве классификатор был выбран алгоритм CatBoost по нескольким причинам причинам:

* по заверению авторов он хорош;
* алгоритм обладает малым набором гиперпараметров в существенной мере влияющих на качество работы;
* реализация обладает удобным интерфейсом;
* по факту, он действительно хорошо себя показал.

Классификатор запускался на следующих признаках:

* язык (кат.);
* страна (кат.);
* класс фильма (rating) (кат.);
* векторное представление постера размерности 64.

Перебор гиперпараметров был выполнен при помощи перекрестной проверки с 5-частным разбиением. Главным критерием было показание метрики ROC AUC (именно на тестовой части, состоавляющей 1/5 от обучающей выборки).

##### 1.3 Построение финальных предсказаний
Финальные предсказания строились по ансамблю моделей, полученных в ходе проведения перекрестной проверки, путем равноправного усреднения их результатов.

### 2. Неуспешные и неиспробованные подходы
1. Одной из идей было применение OCR для извлечения текстовой информации из постера фильма. Вариантами применений такой информации могли стать:
    1. Добавление языковой модели в финальное решение (в виде классификатора поверх countvec или нейросетевой рекуррентной модели).
    2. Поиск по полной базе IMBD по названиям.
    
  Идею реализовать не получилось. Были испробованы различные движки OCR (в том числе tesseract). Качество их работы было крайне плохим. Полагаю, основной причиной неудачи стало низкое разрешение изображений.
  
2. Детектирование лиц и последующая классификация их выражений.
3. Детектирование всевозможных объектов присутствующих на постере при помощи, например, YoLo и добавление полученной информации в качестве признаков для бинарного классификатора.

__За идеями 2 и 3 по моему мнению большой потенциал, но проверять их работоспособность на изображениях столь низкого разрешения показалось тратой времени.__

In [1]:
import os
import shutil
import time

import numpy as np
import pandas as pd
from PIL import Image
from catboost import CatBoostClassifier
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from metrics import f1, precision, recall
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from tqdm import tqdm_notebook

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [82]:
cat_colomns = ['Language', 'Country', 'Rating']

df = pd.read_csv('../data/imdb_wg/train.csv')
leak_df = pd.read_csv('../data/imdb_wg/test_w_labels.csv')

test_ids = leak_df['Id'].values

poster_ids = df['Poster'].values
leak_poster_ids = leak_df['Poster']

df.drop(['Id', 'Poster'], axis=1, inplace=True)
leak_df.drop(['Id', 'Poster'], axis=1, inplace=True)

df.fillna(-999, inplace=True)
leak_df.fillna(-999, inplace=True)

X, y = df.drop('Target', axis=1).as_matrix(), df['Target'].values
X_leak, y_leak = leak_df.drop('Target', axis=1).as_matrix(), leak_df['Target'].values

## Подготовим разбиение train данных на 5 фолдов

In [3]:
fold_idxs = list(StratifiedKFold(y, n_folds=5, shuffle=True))

In [4]:
import pickle

# with open('spilts.p', 'wb') as f:
#     pickle.dump(fold_idxs, f)

with open('spilts.p', 'rb') as f:
    fold_idxs = pickle.load(f)

In [6]:
def load_posters(ids):
    posters = []

    for poster_id in tqdm_notebook(ids):
        poster_path = os.path.join('../data/imdb_wg/posters/', poster_id)
        posters.append(np.array(Image.open(poster_path).convert('RGB')).astype(np.float32))

    return np.array(posters)

In [7]:
posters = load_posters(poster_ids)
leak_posters = load_posters(leak_poster_ids)

In [8]:
def calc_mean_std(X):
    mean, std = np.zeros((3,)), np.zeros((3,))
    
    for x in X:
        x = x / 255.
        x = x.reshape(-1, 3)
        mean += x.mean(axis=0)
        std += x.std(axis=0)
        
    return mean / len(X), std / len(X)

def preprocess_poster(x, train_mean, train_std):
    x = x / 255.
    x -= train_mean
    x /= train_std
    return x

def get_inceptionV3_model():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(268, 182, 3))
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(512, activation='relu')(x)

    x = Dropout(0.5)(x)

    x = Dense(64)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    return Model(inputs=base_model.input, outputs=predictions)

def get_vgg16_model():
    base_model = VGG16(include_top=False, input_shape=(268, 182, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(64)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return Model(inputs=base_model.input, outputs=predictions)

def pop(self):
    '''Removes a layer instance on top of the layer stack.'''
    if not self.outputs:
        raise Exception('Sequential model cannot be popped: model is empty.')
    else:
        self.layers.pop()
        if not self.layers:
            self.outputs = []
            self.inbound_nodes = []
            self.outbound_nodes = []
        else:
            self.layers[-1].outbound_nodes = []
            self.outputs = [self.layers[-1].output]
        self.built = False
        
def pop_last_3_layers(model):
    pop(model)
    pop(model)
    pop(model)
    return model

In [9]:
batch_size = 32
checkpoints_prefix = '/media/e/wg_v2/'

!mkdir '/media/e/wg_v2/'

mkdir: cannot create directory ‘/media/e/wg_v2/’: File exists


## Учим по сетке для каждого фолда (EarlyStop(patience=10) по val_loss)

In [10]:
train_stat = []

for fold_i, (train_idxs, test_idxs) in tqdm_notebook(list(enumerate(fold_idxs))):
    fold_stat = {}
    
    # разбиваем данные
    y_train, y_test = y[train_idxs], y[test_idxs]
    train_posters, test_posters = posters[train_idxs], posters[test_idxs]
    
    # подготовка входов сети
    train_mean, train_std = calc_mean_std(train_posters)
    
    fold_stat['posters_train_mean'] = train_mean
    fold_stat['posters_train_std'] = train_std

    train_datagen = ImageDataGenerator(
        shear_range=0.4,
        horizontal_flip=True,
        rotation_range=20.,
        width_shift_range=0.4,
        height_shift_range=0.4,
        zoom_range=0.4,
        vertical_flip=True,
    )
    
    X_train_posters = np.array([preprocess_poster(poster_i, train_mean, train_std)
                               for poster_i in train_posters])

    train_datagen.fit(X_train_posters, augment=True)
    train_flow = train_datagen.flow(X_train_posters, y_train, batch_size=batch_size)

    
    X_test_posters = np.array([preprocess_poster(poster_i, train_mean, train_std)
                               for poster_i in test_posters])

    steps_per_epoch = len(X_train_posters) // batch_size
    
    # подготовка модели для обучения только новых слоев
    model = get_vgg16_model()
    
    for layer in model.layers[:-5]:
        layer.trainable = False
    for layer in model.layers[-5:]:
        layer.trainable = True

    model.compile(optimizer='rmsprop', loss='binary_crossentropy',
                  metrics=['binary_accuracy', precision, recall, f1])
    
    base_checkpoints_path = os.path.join(checkpoints_prefix, 'base_checkpoints', str(fold_i))
    checkpoints_path = os.path.join(checkpoints_prefix, 'checkpoints', str(fold_i))
    
    shutil.rmtree(base_checkpoints_path, ignore_errors=True)
    shutil.rmtree(checkpoints_path, ignore_errors=True)
    
    os.makedirs(base_checkpoints_path)
    os.makedirs(checkpoints_path)

    base_checkpoints_path_template = os.path.join(base_checkpoints_path, 'base_checkpoint.hdf5')
    checkpoints_path_template = os.path.join(checkpoints_path, 'checkpoints.hdf5')
    
    base_checkpointer = ModelCheckpoint(base_checkpoints_path_template, save_best_only=True)
    checkpointer = ModelCheckpoint(checkpoints_path_template, save_best_only=True)
    
    base_history = model.fit_generator(
        train_flow,
        steps_per_epoch=steps_per_epoch,
        validation_data=(X_test_posters, y_test),
        epochs=10,
        callbacks=[TQDMNotebookCallback(leave_outer=False), base_checkpointer, EarlyStopping(patience=10)],
        verbose=0
    )
    
    fold_stat['base_history'] = base_history.history
    
    # обучение пары последних сверточных блоков
    for layer in model.layers[:15]:
        layer.trainable = False
    for layer in model.layers[15:]:
        layer.trainable = True

    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy',
                  metrics=['binary_accuracy', precision, recall, f1])
    
    history = model.fit_generator(
        train_flow,
        steps_per_epoch=steps_per_epoch,
        validation_data=(X_test_posters, y_test),
        epochs=75,
        callbacks=[TQDMNotebookCallback(leave_outer=False), checkpointer, EarlyStopping(patience=10)],
        verbose=0
    )
    
    fold_stat['history'] = history.history
    
    # получение векторных представлений постеров
    model_path = os.path.join(checkpoints_path, 'checkpoints.hdf5')
    model = load_model(model_path, compile=False)
    model = pop_last_3_layers(model)
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

    fold_stat['train_posters_vect_repr'] = model.predict(X_train_posters)
    fold_stat['test_posters_vect_repr'] = model.predict(X_test_posters)
    
    train_stat.append(fold_stat)

In [5]:
import pickle

# with open('nn_train_stats_2.p', 'wb') as f:
#     pickle.dump(train_stat, f)

with open('nn_train_stats_2.p', 'rb') as f:
    train_stat = pickle.load(f)

In [16]:
import matplotlib.pyplot as plt
%matplotlib notebook

def plot_fold(fold_i, start_epoch=None, end_epoch=None, history_type='last'):
    if history_type == 'last':
        history = 'history'
    else:
        history = 'base_history'
        
    df = pd.DataFrame(train_stat[fold_i][history])
    
    start_epoch = start_epoch or 0
    end_epoch = end_epoch or len(df)
    
    df = df.iloc[start_epoch:end_epoch]
    
    for metric in ['loss', 'precision', 'recall', 'f1']:
        df[[metric, 'val_{}'.format(metric)]].plot()
        plt.show()

## Строим векторные представления для тестовых постеров

In [18]:
leak_posters_vect = []

for fold_i, _ in tqdm_notebook(list(enumerate(fold_idxs))):
    fold_stat = train_stat[fold_i]
    
    train_mean, train_std = fold_stat['posters_train_mean'], fold_stat['posters_train_std']
    
    X_leak_posters = np.array([preprocess_poster(poster_i, train_mean, train_std)
                               for poster_i in leak_posters])
    
    checkpoints_path = os.path.join(checkpoints_prefix, 'checkpoints', str(fold_i))
    
    model_path = os.path.join(checkpoints_path, 'checkpoints.hdf5')
    model = load_model(model_path, compile=False)
    model = pop_last_3_layers(model)
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')
    
    leak_posters_vect.append(model.predict(X_leak_posters))

In [6]:
# with open('leak_posters_vect_2.p', 'wb') as f:
#     pickle.dump(leak_posters_vect, f)

In [7]:
with open('leak_posters_vect_2.p', 'rb') as f:
    leak_posters_vect = pickle.load(f)

## Учим CatBoost (5-fold CV)

In [66]:
# params_space = {
#     'l2_leaf_reg': [1, 3, 5, 7],
#     'learning_rate': [0.01],
#     'depth': [3, 6, 12],
#     'random_strength': [0.5, 0.95, 1],
#     'loss_function': ['CrossEntropy']
# }

params_space = {
    'l2_leaf_reg': [7],
    'learning_rate': [0.01],
    'depth': [12],
    'random_strength': [0.95],
    'loss_function': ['CrossEntropy']
}

In [67]:
# cat_features = [1, 2, 3]
cat_features = list(range(1, X.shape[1]))
cat_boost_train_stat = []

for params in tqdm_notebook(ParameterGrid(params_space)):
    params_stat = {
        'leak_preds': [],
        'test_preds': [],
        'test_scores': [],
        'leak_scores': [],
        'models': []
    }
    start = time.time()

    try:
        for fold_i, (train_idxs, test_idxs) in list(enumerate(fold_idxs)):
            fold_stat = train_stat[fold_i]

            X_train, X_test = X[train_idxs], X[test_idxs]
            y_train, y_test = y[train_idxs], y[test_idxs]

            X_posters_train, X_posters_test = fold_stat['train_posters_vect_repr'], fold_stat['test_posters_vect_repr']

            X_train = np.concatenate((X_train, X_posters_train), axis=1)
            X_test = np.concatenate((X_test, X_posters_test), axis=1)

            X_leak_posters = leak_posters_vect[fold_i]
            X_leak_ = np.concatenate((X_leak, X_leak_posters), axis=1)
            y_leak_ = y_leak


            clf = CatBoostClassifier(
                l2_leaf_reg=int(params['l2_leaf_reg']),
                learning_rate=params['learning_rate'],
                depth=params['depth'],
                random_strength=params['random_strength'],
                iterations=1000,
                eval_metric='AUC',
                use_best_model=True,
                random_seed=42
            )
            clf = clf.fit(
                X_train, y_train,
                cat_features=cat_features,
                eval_set=(X_test, y_test),
                use_best_model=True,
            )
            
            params_stat['models'].append(clf)

            y_pred_proba = clf.predict_proba(X_test)[:,1]
            y_leak_pred_proba = clf.predict_proba(X_leak_)[:,1]
            
            params_stat['test_preds'].append(y_pred_proba)
            params_stat['leak_preds'].append(y_leak_pred_proba)

            params_stat['test_scores'].append(roc_auc_score(y_test, y_pred_proba))
            params_stat['leak_scores'].append(roc_auc_score(y_leak_, y_leak_pred_proba))

        params['mean_test_score'] = np.mean(params_stat['test_scores'])
        params['mean_leak_score'] = np.mean(params_stat['leak_scores'])

        end = time.time()
        
        params['elapsed'] = end - start
        
        if params['mean_leak_score']:
            print(params)
        params.update(params_stat)

        cat_boost_train_stat.append(params)
    except Exception as e:
        print(e)
        print('-' * 20)
        print('Error')
        print(params)
        print('-' * 20)

{'loss_function': 'CrossEntropy', 'mean_leak_score': 0.92909127706941919, 'learning_rate': 0.01, 'random_strength': 0.95, 'elapsed': 2546.631593942642, 'l2_leaf_reg': 7, 'mean_test_score': 0.91610687430432114, 'depth': 12}



In [68]:
# {'loss_function': 'CrossEntropy', 'mean_leak_score': 0.92909127706941919,
#  'learning_rate': 0.01, 'random_strength': 0.95, 'elapsed': 2575.7580795288086,
#  'l2_leaf_reg': 7, 'mean_test_score': 0.91610687430432114, 'depth': 12}

In [69]:
with open('catboost_train_stat_4.p', 'wb') as f:
    pickle.dump(cat_boost_train_stat, f)

In [47]:
with open('catboost_train_stat_4.p', 'rb') as f:
    cat_boost_train_stat = pickle.load(f)

## Построение финальной посылки
(по факту, лучшие комбинации гиперпараметров для тестовых данных и по кроссвалидации совпадают)

In [71]:
best_cat_boost = max(cat_boost_train_stat, key=lambda x: x['mean_leak_score'])

In [72]:
best_cat_boost['leak_scores']

[0.92786379275450315,
 0.93203298927342648,
 0.92950313701679821,
 0.92746913580246915,
 0.92858733049989883]

In [80]:
best_cat_boost['test_scores']

[0.89736128826530615,
 0.9269393241167434,
 0.91487455197132617,
 0.91243439580133123,
 0.92892481136689842]

In [89]:
mean_pred = np.vstack(best_cat_boost['leak_preds']).T.mean(axis=1)
roc_auc_score(y_leak, mean_pred)

0.93234163124873515

In [83]:
pd.DataFrame({'Id': test_ids, 'Probability': mean_pred}) \
    .to_csv('Комаровский_Дмитрий_Юрьевич_task_2_prediction.csv')

### проверка

In [84]:
!head -n 10 Комаровский_Дмитрий_Юрьевич_task_2_prediction.csv

,Id,Probability
0,3636,0.6826362192380488
1,3637,0.1089410532512501
2,3638,0.9635501557636095
3,3639,0.010309313590983575
4,3640,0.03985107248082752
5,3641,0.808477982368165
6,3642,0.030830864208894648
7,3643,0.7757828535866154
8,3644,0.13606035518935988


In [85]:
!wc -l Комаровский_Дмитрий_Юрьевич_task_2_prediction.csv

910 Комаровский_Дмитрий_Юрьевич_task_2_prediction.csv
